# MLP

### load data

In [200]:
from sklearn.neural_network import MLPClassifier
import numpy as np
import pandas as pd
from datetime import datetime
from functools import reduce
import operator 
import matplotlib.pyplot as plt
%matplotlib inline

def str_bool(x):
    if x == 'True':
        x=1
    elif x== 'False':
        x=0
    return x
        

In [250]:
gt_hashtag_lang= pd.read_csv('data/groundtruth_hashtags_lang.csv',index_col=0 )
gt_hashtag_lang=gt_hashtag_lang[~gt_hashtag_lang['tweet_id'].isna()]
gt_hashtag_lang=gt_hashtag_lang.set_index('tweet_id')

gt_hashtag_lang['quoted_status']= ~gt_hashtag_lang['quoted_status'].isna()
gt_hashtag_lang=gt_hashtag_lang.drop( [ 'lang', 'text', 'user_lang','retweeted'], axis =1)


lang_col = [col for col in gt_hashtag_lang.columns if col.startswith('lang_')]
user_lang_col= [col for col in gt_hashtag_lang.columns if col.startswith('user_lang_')]
hashtag_col= [col for col in gt_hashtag_lang.columns if col.startswith('Hashtag topic')]

gt_hashtag_lang[[ 'quoted_status']] = gt_hashtag_lang[[ 'quoted_status']].astype(int)
#gt_hashtag_lang['retweeted'] = gt_hashtag_lang['retweeted'].apply(lambda x: str_bool(x))
gt_hashtag_lang[user_lang_col + ['other user lang']] = gt_hashtag_lang[user_lang_col + ['other user lang']].fillna(0).astype(int)
gt_hashtag_lang[lang_col + ['other lang']] = gt_hashtag_lang[lang_col + ['other lang']].fillna(0).astype(int)
gt_hashtag_lang[hashtag_col] = gt_hashtag_lang[hashtag_col].fillna(0).apply(lambda x: [str_bool(y) for y in x.values])
gt_hashtag_lang[hashtag_col] = gt_hashtag_lang[hashtag_col].astype(float)
gt_hashtag_lang


,hashtags,quoted_status,label,Hashtag topic 0,Hashtag topic 1,Hashtag topic 2,Hashtag topic 3,Hashtag topic 4,Hashtag topic 5,Hashtag topic 6,...,user_lang_de,user_lang_en-gb,user_lang_ar,lang_fr,lang_en,lang_und,lang_es,lang_pt,other lang,other user lang
tweet_id,,,,,,,,,,,,,,,,,,,,,
55c272820a1776527f63ad53,wakeupman,0,1.0,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,...,0,0,0,1,0,0,0,0,0,0
55c3bfea0a1776527f63df5b,NaN,0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0,0,1,0,0,0,0,0,0
55c5cf6f0a1776527f641ef3,NaN,0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0,0,1,0,0,0,0,0,0
55c63c020a1776527f6430bd,NaN,0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0,0,1,0,0,0,0,0,0
55c694b10a1776527f643f13,NaN,0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0,1,0,0,0,0,0,1,0
55e48cd80a17763f6b9f04b0,NaN,0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0,0,1,0,0,0,0,0,0
55e8b92c0a17763f6b9f9e07,NaN,0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0,0,1,0,0,0,0,0,0
55e8bb0e0a17763f6b9f9ea5,NaN,0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0,0,1,0,0,0,0,0,0
55e960ef0a17763f6b9fac6f,NaN,0,1.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0,0,0,1,0,0,0,0,0,0


## Model with 19 features (no hashtags, no images )

In [251]:
gt_20=gt_hashtag_lang[gt_hashtag_lang['hashtags'].isna()]
gt_20=gt_20.drop(['Hashtag topic 0', 'Hashtag topic 1', 'Hashtag topic 2',
       'Hashtag topic 3', 'Hashtag topic 4', 'Hashtag topic 5',
       'Hashtag topic 6', 'Hashtag topic 7', 'hashtags'], axis= 1)

gt_20.head()

,quoted_status,label,user_lang_fr,user_lang_en,user_lang_de,user_lang_en-gb,user_lang_ar,lang_fr,lang_en,lang_und,lang_es,lang_pt,other lang,other user lang
tweet_id,,,,,,,,,,,,,,
55c3bfea0a1776527f63df5b,0,1.0,1,0,0,0,0,1,0,0,0,0,0,0
55c5cf6f0a1776527f641ef3,0,1.0,1,0,0,0,0,1,0,0,0,0,0,0
55c63c020a1776527f6430bd,0,1.0,1,0,0,0,0,1,0,0,0,0,0,0
55c694b10a1776527f643f13,0,1.0,0,0,0,0,1,0,0,0,0,0,1,0
55e48cd80a17763f6b9f04b0,0,1.0,1,0,0,0,0,1,0,0,0,0,0,0


In [252]:
gt_20 = gt_20.sample(frac=1)
train_20=gt_20.iloc[0:int(round(0.5*gt_20.shape[0]))]
val_20=gt_20.iloc[ int(round(0.5*gt_20.shape[0])): int(round(0.75*gt_20.shape[0]))]
test_20=gt_20.iloc[int(round(0.75*gt_20.shape[0])): int(gt_20.shape[0])]

### MLP

In [253]:
mlp_20= MLPClassifier(activation ='relu', solver='adam') #alpha=0.0001 #batch size =200 , max_iter 200
mlp_20.fit(train_20.drop('label',axis=1), train_20['label'])
score = mlp_20.score(test_20.drop('label', axis =1),test_20['label'])
print(score)

0.6319093553820242


## Model with 24 features (hashtags)

In [254]:
gt_hashtag=gt_hashtag_lang[~gt_hashtag_lang['hashtags'].isna()]
gt_hashtag=gt_hashtag.drop('hashtags', axis= 1)
print(gt_hashtag.shape)
gt_hashtag.head()

(8398, 22)


,quoted_status,label,Hashtag topic 0,Hashtag topic 1,Hashtag topic 2,Hashtag topic 3,Hashtag topic 4,Hashtag topic 5,Hashtag topic 6,Hashtag topic 7,...,user_lang_de,user_lang_en-gb,user_lang_ar,lang_fr,lang_en,lang_und,lang_es,lang_pt,other lang,other user lang
tweet_id,,,,,,,,,,,,,,,,,,,,,
55c272820a1776527f63ad53,0,1.0,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,...,0,0,0,1,0,0,0,0,0,0
560311510a17765b698feda1,0,1.0,0.0625,0.0625,0.0625,0.0625,0.0625,0.0625,0.5625,0.0625,...,0,0,0,1,0,0,0,0,0,0
5603eb0d0a177668eb3c3e52,0,1.0,0.0625,0.0625,0.0625,0.5625,0.0625,0.0625,0.0625,0.0625,...,0,0,0,1,0,0,0,0,0,0
561c9a790a1776524a6a96b5,0,1.0,0.0625,0.0625,0.0625,0.5625,0.0625,0.0625,0.0625,0.0625,...,0,0,0,1,0,0,0,0,0,0
5635eb7d0a17762825c218b5,0,1.0,0.0625,0.0625,0.0625,0.5625,0.0625,0.0625,0.0625,0.0625,...,0,0,0,1,0,0,0,0,0,0


In [255]:
gt_hashtag.columns

Index([u'quoted_status', u'label', u'Hashtag topic 0', u'Hashtag topic 1',
       u'Hashtag topic 2', u'Hashtag topic 3', u'Hashtag topic 4',
       u'Hashtag topic 5', u'Hashtag topic 6', u'Hashtag topic 7',
       u'user_lang_fr', u'user_lang_en', u'user_lang_de', u'user_lang_en-gb',
       u'user_lang_ar', u'lang_fr', u'lang_en', u'lang_und', u'lang_es',
       u'lang_pt', u'other lang', u'other user lang'],
      dtype='object')

In [256]:
gt_hashtag = gt_hashtag.sample(frac=1)
train_hashtag=gt_hashtag.iloc[0:int(round(0.5*gt_hashtag.shape[0]))]
val_hashtag=gt_hashtag.iloc[ int(round(0.5*gt_hashtag.shape[0])): int(round(0.75*gt_hashtag.shape[0]))]
test_hashtag=gt_hashtag.iloc[int(round(0.75*gt_hashtag.shape[0])): int(gt_hashtag.shape[0])]

In [257]:
train_hashtag

,quoted_status,label,Hashtag topic 0,Hashtag topic 1,Hashtag topic 2,Hashtag topic 3,Hashtag topic 4,Hashtag topic 5,Hashtag topic 6,Hashtag topic 7,...,user_lang_de,user_lang_en-gb,user_lang_ar,lang_fr,lang_en,lang_und,lang_es,lang_pt,other lang,other user lang
tweet_id,,,,,,,,,,,,,,,,,,,,,
58048d280a17763c56ac46ef,0,0.0,0.062500,0.062500,0.062500,0.062500,0.062500,0.062500,0.562500,0.062500,...,0,0,0,0,1,0,0,0,0,0
5721d4060a17762e8f8e4509,0,0.0,0.062500,0.562500,0.062500,0.062500,0.062500,0.062500,0.062500,0.062500,...,0,0,0,0,1,0,0,0,0,0
57b4f75d0a17765c3e9693ea,1,0.0,0.562500,0.062500,0.062500,0.062500,0.062500,0.062500,0.062500,0.062500,...,0,0,0,0,1,0,0,0,0,0
574ef29c0a17762e8f908c29,0,1.0,0.041667,0.041667,0.041667,0.041667,0.041667,0.041667,0.041667,0.708333,...,0,1,0,0,1,0,0,0,0,0
5733807f0a17762e8f8f0878,0,0.0,0.062500,0.562500,0.062500,0.062500,0.062500,0.062500,0.062500,0.062500,...,0,0,0,0,1,0,0,0,0,0
584c170c0a1776133330ab43,0,1.0,0.017857,0.160712,0.017857,0.017857,0.017857,0.017857,0.017857,0.732145,...,0,0,0,0,1,0,0,0,0,0
57b60adc0a17765c3e969d22,0,1.0,0.041667,0.708333,0.041667,0.041667,0.041667,0.041667,0.041667,0.041667,...,0,0,0,0,1,0,0,0,0,0
57b4f78e0a17765c3e9693ed,1,0.0,0.562500,0.062500,0.062500,0.062500,0.062500,0.062500,0.062500,0.062500,...,0,0,0,0,1,0,0,0,0,0
5775a6e70a17764baf42fdf0,0,0.0,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,...,0,0,0,1,0,0,0,0,0,0


In [264]:
mlp_hashtag= MLPClassifier(activation ='relu', solver='adam') #alpha=0.0001 #batch size =200 , max_iter 200
mlp_hashtag.fit(train_hashtag.drop('label',axis=1).fillna(0), train_hashtag['label'].fillna(0).astype(int))
score_hashtag = mlp_hashtag.score(test_hashtag.drop('label', axis =1).fillna(0),test_hashtag['label'].fillna(0).astype(int))
print(score)

0.6319093553820242
